# Intraclass Correlation Coefficient (ICC)

In [1]:
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_theme(
    context="paper", 
    style="whitegrid", 
    font_scale=1.2,
    rc={'figure.figsize': (10, 10), 'figure.dpi': 300}
)

In [7]:
import pingouin as pg
from common.data import get_model_weights
from common.wisc import WISC_LEVEL
from common.binning import BIN_LABELS
from common.paths import MODEL_RESULTS, ICC_RESULTS
from common.results import save_results

## Between PLS and Ridge

In [4]:
%%time
results = []
population = 'adhd'

for bin_label in BIN_LABELS:
    for target in WISC_LEVEL[5]:
        pls_weights = get_model_weights('pls', population, target, bin_label)
        ridge_weights = get_model_weights('ridge', population, target, bin_label)
        
        icc_data = pd.DataFrame([pls_weights, ridge_weights]).melt(
            var_name='connection', value_name='weight', ignore_index=False)
        icc_data['cv_run_num'] = icc_data.index
        icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
        icc1 = icc[icc['Type'] == 'ICC1'].to_dict('records')[0]
        
        results.append({
            'Population': population,
            'Target': target,
            'Bin': bin_label,
            **icc1,
        })
#         print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df)

,Population,Target,Bin,Type,Description,ICC,F,df1,df2,pval,CI95%
0,adhd,WISC_FSIQ,All,ICC1,Single raters absolute,0.959,47.922,34715,34716,0.0,"[0.96, 0.96]"


CPU times: user 34.4 s, sys: 184 ms, total: 34.6 s
Wall time: 34.6 s


In [ ]:
filename = f'between-model_icc_{population}.csv'
save_results(results_df, filename, ICC_RESULTS)

## Between age bins

In [3]:
import itertools

all_bin_combinations = list(itertools.combinations(BIN_LABELS, 2))
print(all_bin_combinations)

[('All', 'Bin 1'), ('All', 'Bin 2'), ('All', 'Bin 3'), ('Bin 1', 'Bin 2'), ('Bin 1', 'Bin 3'), ('Bin 2', 'Bin 3')]


In [5]:
%%time
results = []
population = 'adhd'
model = 'ridge'

for target in WISC_LEVEL[5]:
    for bin_pair in all_bin_combinations:
        first_bin, second_bin = bin_pair
        first_bin_weights = get_model_weights(model, population, target, first_bin)
        second_bin_weights = get_model_weights(model, population, target, second_bin)
        
        icc_data = pd.DataFrame([first_bin_weights, second_bin_weights]).melt(
            var_name='connection', value_name='weight', ignore_index=False)
        icc_data['cv_run_num'] = icc_data.index
        icc = pg.intraclass_corr(data=icc_data, targets='connection', raters='cv_run_num', ratings='weight').round(3)
        icc1 = icc[icc['Type'] == 'ICC1'].to_dict('records')[0]
        
        results.append({
            'Model': model,
            'Population': population,
            'Target': target,
            'First Bin': first_bin,
            'Second Bin': second_bin,
            **icc1,
        })
        print(results[-1])
        
results_df = pd.DataFrame(results)
display(results_df)

{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'First Bin': 'All', 'Second Bin': 'Bin 1', 'Type': 'ICC1', 'Description': 'Single raters absolute', 'ICC': 0.512, 'F': 3.101, 'df1': 34715, 'df2': 34716, 'pval': 0.0, 'CI95%': array([0.5 , 0.52])}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'First Bin': 'All', 'Second Bin': 'Bin 2', 'Type': 'ICC1', 'Description': 'Single raters absolute', 'ICC': 0.582, 'F': 3.78, 'df1': 34715, 'df2': 34716, 'pval': 0.0, 'CI95%': array([0.57, 0.59])}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'First Bin': 'All', 'Second Bin': 'Bin 3', 'Type': 'ICC1', 'Description': 'Single raters absolute', 'ICC': 0.596, 'F': 3.946, 'df1': 34715, 'df2': 34716, 'pval': 0.0, 'CI95%': array([0.59, 0.6 ])}
{'Model': 'ridge', 'Population': 'adhd', 'Target': 'WISC_FSIQ', 'First Bin': 'Bin 1', 'Second Bin': 'Bin 2', 'Type': 'ICC1', 'Description': 'Single raters absolute', 'ICC': 0.107, 'F': 1.241, 'df1': 34715, 'df2': 34716

,Model,Population,Target,First Bin,Second Bin,Type,Description,ICC,F,df1,df2,pval,CI95%
0,ridge,adhd,WISC_FSIQ,All,Bin 1,ICC1,Single raters absolute,0.512,3.101,34715,34716,0.000,"[0.5, 0.52]"
1,ridge,adhd,WISC_FSIQ,All,Bin 2,ICC1,Single raters absolute,0.582,3.780,34715,34716,0.000,"[0.57, 0.59]"
2,ridge,adhd,WISC_FSIQ,All,Bin 3,ICC1,Single raters absolute,0.596,3.946,34715,34716,0.000,"[0.59, 0.6]"
3,ridge,adhd,WISC_FSIQ,Bin 1,Bin 2,ICC1,Single raters absolute,0.107,1.241,34715,34716,0.000,"[0.1, 0.12]"
4,ridge,adhd,WISC_FSIQ,Bin 1,Bin 3,ICC1,Single raters absolute,0.079,1.171,34715,34716,0.000,"[0.07, 0.09]"
5,ridge,adhd,WISC_FSIQ,Bin 2,Bin 3,ICC1,Single raters absolute,0.076,1.164,34715,34716,0.000,"[0.07, 0.09]"
6,ridge,adhd,WISC_VSI,All,Bin 1,ICC1,Single raters absolute,0.440,2.570,34715,34716,0.000,"[0.43, 0.45]"
7,ridge,adhd,WISC_VSI,All,Bin 2,ICC1,Single raters absolute,0.565,3.599,34715,34716,0.000,"[0.56, 0.57]"
8,ridge,adhd,WISC_VSI,All,Bin 3,ICC1,Single raters absolute,0.501,3.005,34715,34716,0.000,"[0.49, 0.51]"
9,ridge,adhd,WISC_VSI,Bin 1,Bin 2,ICC1,Single raters absolute,0.127,1.290,34715,34716,0.000,"[0.12, 0.14]"


CPU times: user 20min 1s, sys: 5.56 s, total: 20min 7s
Wall time: 20min 6s


In [8]:
filename = f'between-age-bins_icc_{population}.csv'
save_results(results_df, filename, ICC_RESULTS)

'/home/bpho/Documents/MSc_Research-Project/scratch_data/model_results/ICC/between-age-bins_icc_adhd.csv.csv'

### Visualize results